# CEIA - NLP Desafío N°1
## Villanueva Cecilia Azul

### Librerías

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd

### Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.


#### 1 - Resolución: Vectorización de Documentos

In [120]:
#Vectorizacion
tfidfvect_1 = TfidfVectorizer()

# `X_train` matriz documento-término
X_train = tfidfvect_1.fit_transform(newsgroups_train.data)

# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target


In [121]:
# Semilla para reproducibilidad
np.random.seed(42)

In [122]:
# Seleccionamos 5 documentos aleatorios
indices = np.random.choice(X_train.shape[0], size=5, replace=False)

# Iteramos sobre los índices de los documentos aleatorios
for idx in indices:
    print("=" * 80)
    print(f"Documento Aleatorio (Index {idx})")
    print(f"Clase del Documento: {newsgroups_train.target_names[y_train[idx]]}")
    print("Contenido del Documento:")
    print(newsgroups_train.data[idx][:500])  # Muestra los primeros 500 caracteres del contenido
    print()
    # Calculamos la similitud de coseno con todos los documentos de entrenamiento
    cossim = cosine_similarity(X_train[idx], X_train)[0]

    # Ordenamos los valores de similitud de mayor a menor
    most_similar_indices = np.argsort(cossim)[::-1][1:6]
    most_similar_values = np.sort(cossim)[::-1][1:6]

    # Imprimimos los 5 documentos más similares
    for i, sim_idx in enumerate(most_similar_indices):
        print("-" * 80)
        print(f"{i + 1} - Documento más similar")
        print(f"Index: {sim_idx}")
        print(f"Clase del Documento: {newsgroups_train.target_names[y_train[sim_idx]]}")
       # print(f"Contenido del Documento:")
       # print(newsgroups_train.data[sim_idx][:500])  # Muestra los primeros 500 caracteres del contenido
        print(f"Similitud: {most_similar_values[i]:.4f}")
        print("---------------------------------------------------")
        print()


Documento Aleatorio (Index 7492)
Clase del Documento: comp.sys.mac.hardware
Contenido del Documento:
Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward" 

--------------------------------------------------------------------------------
1 - Documento más similar
Index: 10935
Clase del Documento: comp.sys.mac.hardware
Similitud: 0.6665
---------------------------------------------------

--------------------------------------------------------------------------------
2 - Documento más similar
Index: 7258
Clase del Documento: comp.sys.ibm.pc.hardware
Similitud: 0.3476
---------------------------------------------------

--------------------------------------------------------------------------------
3 - Documento más similar
Index: 4971
Clase del Documento: comp.s

Se observa que los documentos pertenecientes a las clases de contenido técnico son similares entre sí, esto podría deberse a la presencia de vocabulario técnico.
En el caso documentos pertenecientes a clases vinculadas con politica, religión o deportes se presentan similariades con documentos de otras clases.
A nivel general se puede ver que la similitud tiene sentido en relación al contenido de los documentos.

#### 2 - Resolución: Entrenamiento de un clasificador de texto con Naïve Bayes

In [115]:
# Vectorización con hiperparámetros ajustados
tfidfvect_2 = TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1,2), sublinear_tf=True, norm='l2')


X_train = tfidfvect_2.fit_transform(newsgroups_train.data)
X_test = tfidfvect_2.transform(newsgroups_test.data)

y_train = newsgroups_train.target
y_test = newsgroups_test.target

# Definir modelos
models = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB()
}

# GridSearch para encontrar el mejor alpha
param_grid = {'alpha': [0.1, 0.5, 1.0, 2.0, 5.0]}
best_models = {}

In [116]:
for name, model in models.items():
    grid_search = GridSearchCV(model, param_grid, scoring='f1_macro', cv=5)
    grid_search.fit(X_train, y_train)
    best_models[name] = grid_search.best_estimator_
    print(f"Mejor modelo para {name}: {grid_search.best_params_}")

# Evaluar en test
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return f1_score(y_test, y_pred, average='macro')

for name, model in best_models.items():
    f1 = evaluate_model(model, X_test, y_test)
    print(f"F1-score macro en test para {name}: {f1:.4f}")


Mejor modelo para MultinomialNB: {'alpha': 0.1}
Mejor modelo para ComplementNB: {'alpha': 0.1}
F1-score macro en test para MultinomialNB: 0.6432
F1-score macro en test para ComplementNB: 0.7014


Se observa que modificando los parámetros de la vectorización se obtiene un mejor resultado. A su vez el modelo ComplementNB obtiene un mejor resultado que el MultinomialNB.

#### 3 - Transponer la matriz documento-término

In [117]:
# Vectorización con hiperparámetros ajustados
tfidfvect_3 = TfidfVectorizer(max_df=0.95, min_df=2, ngram_range=(1,1), sublinear_tf=True, norm='l2',  stop_words='english')

X_train = tfidfvect_3.fit_transform(newsgroups_train.data)


In [118]:
# Transponer la matriz documento-término para obtener la matriz término-documento
X_terms = X_train.T

# Obtener el vocabulario
terms = np.array(tfidfvect_3.get_feature_names_out())

tfidfvect_3.vocabulary_.keys()


dict_keys(['wondering', 'enlighten', 'car', 'saw', 'day', 'door', 'sports', 'looked', 'late', '60s', 'early', '70s', 'called', 'bricklin', 'doors', 'really', 'small', 'addition', 'bumper', 'separate', 'rest', 'body', 'know', 'model', 'engine', 'specs', 'years', 'production', 'history', 'info', 'funky', 'looking', 'mail', 'fair', 'number', 'brave', 'souls', 'upgraded', 'si', 'clock', 'oscillator', 'shared', 'experiences', 'poll', 'send', 'brief', 'message', 'detailing', 'procedure', 'speed', 'attained', 'cpu', 'rated', 'add', 'cards', 'adapters', 'heat', 'sinks', 'hour', 'usage', 'floppy', 'disk', 'functionality', '800', 'floppies', 'especially', 'requested', 'summarizing', 'days', 'network', 'knowledge', 'base', 'upgrade', 'haven', 'answered', 'thanks', 'folks', 'mac', 'plus', 'finally', 'gave', 'ghost', 'weekend', 'starting', 'life', '512k', 'way', '1985', 'sooo', 'market', 'new', 'machine', 'bit', 'sooner', 'intended', 'picking', 'powerbook', '160', 'maybe', '180', 'bunch', 'question

In [119]:
# Elegir palabras manualmente
selected_words = ["computer", "car", "religion", "sports", "programs"]


# Encontrar las 5 palabras más similares a cada una
for word in selected_words:
    if word in tfidfvect_3.vocabulary_:
        idx = tfidfvect_3.vocabulary_[word]
        similarities = cosine_similarity(X_terms[idx], X_terms).flatten()
        similar_indices = similarities.argsort()[-6:-1][::-1]  # Tomar las 5 más similares
        similar_words = terms[similar_indices]
        print(f"Palabras más similares a '{word}': {similar_words}")
    else:
        print(f"'{word}' no está en el vocabulario.")


Palabras más similares a 'computer': ['shopper' 'science' 'software' 'drive' 'shops']
Palabras más similares a 'car': ['cars' 'dealer' 'civic' 'owner' 'engine']
Palabras más similares a 'religion': ['religions' 'religious' 'christianity' 'christian' 'christians']
Palabras más similares a 'sports': ['wip' 'berman' 'sportswriting' 'ethical' '3000gt']
Palabras más similares a 'programs': ['ruiter' 'wordprocessors' 'anton' 'schedulers' 'spreadsheets']


Si bien el análisis está basado en las frecuencias de las palabras, se puede observar que existe relación conceptual entre cada palabra y sus 5 más similares.